In [3]:
test_json = {"A": "Disclose the error to the patient and put it in the operative report", "B": "Tell the attending that he cannot fail to disclose this mistake", "C": "Report the physician to the ethics committee", "D": "Refuse to dictate the operative report" }
prompt = ""

for k, v in test_json.items():
    prompt += f"{k}: {v}\n"
print(prompt)

A: Disclose the error to the patient and put it in the operative report
B: Tell the attending that he cannot fail to disclose this mistake
C: Report the physician to the ethics committee
D: Refuse to dictate the operative report



In [14]:
from datasets import load_dataset
import pandas as pd

data = load_dataset("GBaker/MedQA-USMLE-4-options")
test_df = data['test'].to_pandas()

test_df = test_df.head(210)
cnt = 0
fin_df = pd.DataFrame(columns=['question', 'options', 'answer_idx', 'metamap_phrases'])
for i in range(210):
    if fin_df.shape[0] == 200:
        break
    if '?' in test_df.iloc[i]['question']:
        fin_df.loc[cnt] = [test_df.iloc[i]['question'], test_df.iloc[i]['options'], test_df.iloc[i]['answer_idx'], test_df.iloc[i]['metamap_phrases']]
        cnt += 1

fin_df.to_json('dataset/medQA_4.json', orient='records')

In [2]:
import pandas as pd
fin_df = pd.read_json('dataset/medQA_4.json', orient='records')
print(fin_df.shape)
print(fin_df.head())

(200, 4)
                                            question  \
0  A junior orthopaedic surgery resident is compl...   
1  A 67-year-old man with transitional cell carci...   
2  Two weeks after undergoing an emergency cardia...   
3  A 39-year-old woman is brought to the emergenc...   
4  A 35-year-old man comes to the physician becau...   

                                             options answer_idx  \
0  {'A': 'Disclose the error to the patient and p...          B   
1  {'A': 'Inhibition of proteasome', 'B': 'Hypers...          D   
2  {'A': 'Renal papillary necrosis', 'B': 'Choles...          B   
3  {'A': 'Coagulase-positive, gram-positive cocci...          D   
4  {'A': 'Erythromycin ointment', 'B': 'Ketotifen...          B   

                                     metamap_phrases  
0  [junior orthopaedic surgery resident, completi...  
1  [67 year old man, transitional cell carcinoma ...  
2  [Two weeks, emergency cardiac, stenting, unsta...  
3  [year old woman, brought, em

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained("SecurityXuanwuLab/HaS-820m")
model = AutoModelForCausalLM.from_pretrained("SecurityXuanwuLab/HaS-820m").to('cuda:0')
hide_template = """<s>Paraphrase the text:%s\n\n"""
# original_input = "张伟用苹果(iPhone 13)换了一箱好吃的苹果。"
original_input = "A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?"
input_text = hide_template % original_input
inputs = tokenizer(input_text, return_tensors='pt').to('cuda:0')
pred = model.generate(**inputs, max_length=100)
pred = pred.cpu()[0][len(inputs['input_ids'][0]):]
hide_input = tokenizer.decode(pred, skip_special_tokens=True)
print(hide_input)

# output:
# 李华用华为(Mate 20)换了一箱美味的橙子。

A lifting door is convenient for three direction travel, but it also serves as a security measure at a what?


In [3]:
from openai_backTranslation import generate

hidden_output = generate(hide_input, max_tokens=len(hide_input))
# print(hidden_output)

A lifting door is convenient for three direction travel, but it also serves as a security measure at a what?
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 A, a loading dock, B, a parking garage, C, a car wash, D, a car dealership.

I'm gonna say a loading dock.

That is correct.

Yes.

All right, so you're at two for two. You're doing pretty good.

I'm doing pretty good.

All right, question number three. What is the name of the device that is used to measure the amount of light that is reflected off of a surface?

A, a photometer, B, a spectrometer, C, a reflectometer, D, a radiometer.

I'm gonna say a reflectometer.

That is correct.

Yes.

All right, you're doing pretty good. You're at three for three. All right, question number four. What is the name of the device that is used to measure the amount of light that is reflected off of a surface?

A, a photometer, B, a spectrometer, C, a reflectometer, D, a radiometer.

I'm gonna say a reflectometer.

That is correct.

Yes.

All
 A, a loading dock, B, a parking garage, C, a car wash, D, a car dealership.

I'm gonna say a loading dock.

That is correct.

Yes.

All right, so you're at two

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("SecurityXuanwuLab/HaS-820m")
model = AutoModelForCausalLM.from_pretrained("SecurityXuanwuLab/HaS-820m").to('cuda:0')
seek_template = "Convert the text:\n%s\n\n%s\n\nConvert the text:\n%s\n\n"
hide_input = "前天，'2022北京海淀·颐和园经贸合作洽谈会成功举行，各大媒体竞相报道了活动盛况，小李第一时间将昨天媒体报道情况进行了整理。人民日报 中国青年网 国际联合报 北京商报 消费者观察报 上海晚报 杭州日报 海峡晚报 北京日报 北京市电视一台?北京新闻 人民网 手机雅虎网 网易北京 长三角经济网 新京网 中国农业新闻网 北京圆桌 居然有这么多!还有部分媒体将在未来一周陆续发稿，为经洽会点!为海淀点!阅读投诉阅读精选留言加载中以上留言由公众号筛选后显示了解留言功能详情"
hide_output = "2022北京海淀·颐和园经贸合作洽谈会成功举办，各大媒体广泛报道"
original_input = "昨天，’2016苏州吴中·太湖经贸合作洽谈会成功举行，各大媒体竞相报道了活动盛况，小吴第一时间将今天媒体报道情况进行了整理。新华社 中国青年报?中青在线 香港大公报?大公网 香港商报 消费者导报 扬子晚报 江南时报 苏州日报 姑苏晚报 城市商报 苏州广电一套?苏州新闻 新华网 手机凤凰网 网易苏州 长三角城市网 新苏网 中国商务新闻网 苏州圆桌 居然有这么多!还有部分媒体将在今后几天陆续发稿，为经洽会点!为吴中点!阅读投诉阅读精选留言加载中以上留言由公众号筛选后显示了解留言功能详情"
input_text = seek_template % (hide_input, hide_output, original_input)
inputs = tokenizer(input_text, return_tensors='pt').to('cuda:0')
pred = model.generate(**inputs, max_length=512)
pred = pred.cpu()[0][len(inputs['input_ids'][0]):]
original_output = tokenizer.decode(pred, skip_special_tokens=True)
print(original_output)

# output:
# 2016苏州吴中·太湖经贸合作洽谈会成功举办，各大媒体广泛报道


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2016苏州吴中·太湖经贸合作洽谈会成功举办，各大媒体广泛报道


In [6]:
from openai_backTranslation import generate
from dpps.HaS import HaS
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'


has = HaS(device)
test_question = "A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?"
test_json = {"A": "Disclose the error to the patient and put it in the operative report", "B": "Tell the attending that he cannot fail to disclose this mistake", "C": "Report the physician to the ethics committee", "D": "Refuse to dictate the operative report" }
prompt = ""
for k, v in test_json.items():
    prompt += f"{k}: {v}\n"
    print(f"Paraphrased Choice {k}:\n{has.hide(v)}")
    
    
hide_input = has.hide(prompt)
print(f"Hide Input Prompt1: {hide_input}")

hide_input = has.hide(test_question)
print(f"Hide Input Prompt2: {hide_input}")

hide_input = has.custom_hide(test_question, prompt)

print(f"Hide Input Prompt3: {hide_input}")

# hide_output = generate()

# original_input = "华纳兄弟影业（Warner Bro）著名的作品有《蝙蝠侠》系列、《超人》系列、《黑客帝国》系列和《指环王》系列。目前华纳未考虑推出《蝙蝠侠》系列新作。"
# hide_input = has.hide(original_input)
# hide_output = generate(f"Paraphrase the following document:\n{hide_input}\nParaphrased document:\n", max_tokens=20)
# original_output = has.seek(hide_input, hide_output, original_prompt)
# print("Original Input Prompt:" + original_prompt)
# print("Hide Input Prompt:" + hide_input)
# print("Hide Output Prompt:" + hide_output)
# print("Original Output Prompt:" + original_output)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Paraphrased Choice A:
Disclose the error to the patient and put it in the operative report
Paraphrased Choice B:
Tell the attending that he cannot fail to disclose this mistake
Paraphrased Choice C:
Report the physician to the political committee
Paraphrased Choice D:
Reject to establish the operative inspection report.
Hide Input Prompt1: E: Confess the accident to the patient and put it in the procedural record
Hide Input Prompt2: a senior orthopaedic surgery trainee is completing an elbow disc repair with the department chairman as the attending physician. During the case, the trainee inadvertently cuts a meniscus. The meniscus is repaired without complication. The attending tells the trainee that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the trainee to leave this complication out of the operative report. Which of the following is the correct n

AttributeError: 'HaS' object has no attribute 'custom_hide'